In [ ]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    new_words = []
    for word in words:
        new_words.append(word.lower())
    return new_words
    """Convert all characters to lowercase from list of tokenized words"""

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    stop_words = set(stopwords.words("spanish"))
    new_words = [word for word in words if word not in stop_words]
    return new_words

def preprocessing(words):
    words = to_lowercase(words)
    words = replace_numbers(words)
    words = remove_punctuation(words)
    words = remove_non_ascii(words)
    words = remove_stopwords(words)
    return words
df_obj['Textos_espanol'] = df_obj['Textos_espanol'].apply(contractions.fix) #Aplica la corrección de las contracciones
df_obj.head()
df_obj['Textos_Tokenizados'] = df_obj['Textos_espanol'].apply(contractions.fix)  # Aplica la corrección de las contracciones
df_obj['Textos_Tokenizados'] = df_obj['Textos_espanol'].apply(preprocessing)  # Aplica la eliminación del ruido
df_obj['Textos_Tokenizados'] = df_obj['Textos_espanol'].apply(word_tokenize)  # Tokeniza las palabras


df_obj.head()
df_obj["Textos_Tokenizados"].sample(15)
# Cargar el lematizador de palabras en inglés
wordnet_lemmatizer = WordNetLemmatizer()

# Cargar el extractor de raíces (stemmer) de Lancaster para el inglés
lancaster_stemmer = LancasterStemmer()
def stem_words(words):
    """Stem words in list of tokenized words"""
    stems = [lancaster_stemmer.stem(word) for word in words]
    return stems
def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmas = [wordnet_lemmatizer.lemmatize(word) for word in words]
    return lemmas
def stem_and_lemmatize(words):
    stems = stem_words(words)
    lemmas = lemmatize_verbs(words)
    return stems + lemmas

df_obj['Textos_Tokenizados'] = df_obj['Textos_Tokenizados'].apply(stem_and_lemmatize)
df_obj.head()
df_obj['Textos_Tokenizados'] = df_obj['Textos_Tokenizados'].apply(lambda x: ' '.join(map(str, x)))
df_obj
X_data, y_data = df_obj['Textos_Tokenizados'],df_obj['sdg']
y_data = (y_data == '5').astype(int)
y_data
y_data.value_counts()

# Modelo prediccion

Una vez se tienen en cuenta las vectorizaciones como el los modelos de clasificacion y sus matrices de evaluacion se va a tomar el mejor algoritmo y vectorizacion para llevar a cabo las predicciones en el excel sin etiquetas

El modelo que dio mejores resultados segun las metricas de evaluacion fue el vector TF-IDF con el algoritmo de clasificacion SVM, ya que este fue el mejor resultado se va a llevar a cabo la prediccion con este modelo

In [ ]:
tfidf = TfidfVectorizer(tokenizer=word_tokenize, stop_words=stop_words, lowercase=True)
X_tfidf = tfidf.fit_transform(X_train["Textos_espanol"])
print("Vocabulary size:", len(tfidf.vocabulary_))

In [ ]:
# Crea un modelo de Support Vector Machine (SVM)
svm_model = SVC(kernel='linear', random_state=5)

# Entrena el modelo con los vectores de características generados por Doc2Vec
svm_model.fit(X_tfidf, y_train)

# Realiza predicciones en los datos de prueba
y_test_svm_predict = svm_model.predict(tfidf.transform(X_test["Textos_espanol"]))

# Calcula las métricas de evaluación
precision_svm = precision_score(y_test, y_test_svm_predict, average='weighted')
recall_svm = recall_score(y_test, y_test_svm_predict, average='weighted')
f1_svm = f1_score(y_test, y_test_svm_predict, average='weighted')

print("Precision (SVM):", precision_svm)
print("Recall (SVM):", recall_svm)
print("F1 (SVM):", f1_svm)